In [1]:
import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

def build_model(data):
   
    # this is the size of our encoded representations
    encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    original_dim=data.shape[1]
    input_img = Input(shape=(original_dim,))
    # "encoded" is the encoded representation of the input

    encoded = Dense(256, activation='tanh',
                    activity_regularizer=regularizers.l1(2*10e-5))(input_img)
    encoded = Dense(128, activation='tanh',
                    activity_regularizer=regularizers.l1(2*10e-5))(encoded)
    encoded = Dense(32, activation='tanh',
                    activity_regularizer=regularizers.l1(2*10e-5))(encoded)

    decoded = Dense(128, activation='tanh')(encoded)
    decoded = Dense(256, activation='tanh')(decoded)
    decoded = Dense(original_dim, activation='tanh')(decoded)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded)

    # this model maps an input to its encoded representation
    encoder = Model(input_img, encoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    autoencoder.fit(data, data,
                    epochs=10,
                    batch_size=100,
                    shuffle=True)
    return encoder

Using TensorFlow backend.


In [3]:
from nilearn.decomposition import CanICA
def prepare_data(func_filenames):
    canica = CanICA(n_components=20, smoothing_fwhm=6.,
                    memory="nilearn_cache", memory_level=2,
                    threshold=3., verbose=10, random_state=0)
    data=canica.prepare_data(func_filenames)
    return data

In [4]:
from nilearn.connectome import ConnectivityMeasure

def corr(all_time_series):
    connectivity_biomarkers = {}
    conn_measure = ConnectivityMeasure(kind='correlation', vectorize=True)
    connectivity_biomarkers = conn_measure.fit_transform(all_time_series)
    return connectivity_biomarkers

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def classify(train_X,train_Y, test_X, test_Y):
    names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
             "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
             "Naive Bayes", "QDA"]

    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        GaussianProcessClassifier(1.0 * RBF(1.0)),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        MLPClassifier(alpha=1),
        AdaBoostClassifier(),
        GaussianNB(),
        QuadraticDiscriminantAnalysis()]

    scores = []
    
    for name, clf in zip(names, classifiers):
        clf.fit(train_X, train_Y) 
        score=clf.score(test_X,test_Y)
        scores.append(score)
    return scores



    

In [6]:
from sklearn.model_selection import StratifiedKFold

def CV(X,Y):
    
    # define 10-fold cross validation test harness
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    
    # trick. split(Y,Y) instead of split(X,Y) due to X is already concatnated 
    for train,test in kfold.split(Y,Y):   
        #indexing of specific subjects
        train_X = [X[i*20:i*20+19] for i in train]    

        train_Y = [Y[i] for i in train]
        test_Y = [Y[i] for i in test]

        #concat all subjects
        model=build_model(np.vstack(train_X))

        train_D = [model.predict(X[i*20:i*20+19]) for i in train]
        test_D = [model.predict(X[i*20:i*20+19]) for i in test]

        #Release GPU memory after model is used
        from keras import backend as K
        K.clear_session()
        
        train_FC=corr(train_D)
        test_FC=corr(test_D)

        score=classify(train_FC,train_Y, test_FC, test_Y)
        cvscores.append(score)
    return cvscores

In [7]:
from nilearn import datasets

adhd_dataset = datasets.fetch_adhd(n_subjects=40,data_dir='/home/share/TmpData/Qinglin/nilearn_data/')
X = prepare_data(adhd_dataset.func)  # list of 4D nifti files for each subject  
Y = adhd_dataset.phenotypic['adhd']

cvscores=CV(X,Y)


/home/uga_qinglin/Documents/nilearn-deep/nilearn/datasets/func.py:503: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


[MultiNiftiMasker.fit] Loading data from [/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz, /home/share/TmpData/Qinglin/nilearn_data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.g
[MultiNiftiMasker.fit] Computing mask
[MultiNiftiMasker.transform] Resampling mask
[CanICA] Loading data
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0010128/0010128_rest_tshift_RPI_voreg_mni.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0021019/0021019_rest_tshift_RPI_voreg_mni.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0023008/0023008_rest_tshift_RPI_voreg_mni.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/0023012/0023012_rest_tshift_RPI_voreg_mni.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/adhd/data/00

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:686: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Epoch 1/10
608/608 [==============================] - 3s 5ms/step - loss: 8.7017
Epoch 2/10
608/608 [==============================] - 1s 2ms/step - loss: 7.4495
Epoch 3/10
608/608 [==============================] - 1s 2ms/step - loss: 6.7840
Epoch 4/10
608/608 [==============================] - 2s 3ms/step - loss: 6.3865
Epoch 5/10
608/608 [==============================] - 2s 3ms/step - loss: 6.0834
Epoch 6/10
608/608 [==============================] - 2s 3ms/step - loss: 5.8865
Epoch 7/10
608/608 [==============================] - 1s 2ms/step - loss: 5.7119
Epoch 8/10
608/608 [==============================] - 2s 2ms/step - loss: 5.5468
Epoch 9/10
608/608 [==============================] - 1s 2ms/step - loss: 5.3655
Epoch 10/10
608/608 [==============================] - 1s 2ms/step - loss: 5.3263


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:686: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


...

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:686: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Epoch 1/10
608/608 [==============================] - 3s 6ms/step - loss: 8.7533
Epoch 2/10
608/608 [==============================] - 2s 3ms/step - loss: 7.6047
Epoch 3/10
608/608 [==============================] - 2s 3ms/step - loss: 6.9405
Epoch 4/10
608/608 [==============================] - 2s 3ms/step - loss: 6.5062
Epoch 5/10
608/608 [==============================] - 2s 3ms/step - loss: 6.2005
Epoch 6/10
608/608 [==============================] - 2s 3ms/step - loss: 5.9394
Epoch 7/10
608/608 [==============================] - 2s 3ms/step - loss: 5.7549
Epoch 8/10
608/608 [==============================] - 2s 3ms/step - loss: 5.5889
Epoch 9/10
608/608 [==============================] - 2s 3ms/step - loss: 5.4894
Epoch 10/10
608/608 [==============================] - 2s 3ms/step - loss: 5.3548


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:686: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Epoch 1/10
608/608 [==============================] - 3s 6ms/step - loss: 8.6060
Epoch 2/10
608/608 [==============================] - 2s 3ms/step - loss: 7.4351
Epoch 3/10
608/608 [==============================] - 2s 3ms/step - loss: 6.8101
Epoch 4/10
608/608 [==============================] - 2s 3ms/step - loss: 6.4287
Epoch 5/10
608/608 [==============================] - 2s 3ms/step - loss: 6.0640
Epoch 6/10
608/608 [==============================] - 2s 3ms/step - loss: 5.8205
Epoch 7/10
608/608 [==============================] - 2s 3ms/step - loss: 5.7286
Epoch 8/10
608/608 [==============================] - 2s 2ms/step - loss: 5.5225
Epoch 9/10
608/608 [==============================] - 2s 3ms/step - loss: 5.4668
Epoch 10/10
608/608 [==============================] - 2s 3ms/step - loss: 5.2810


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:686: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [8]:
cvscores

[[0.5, 0.5, 0.5, 0.875, 0.375, 0.25, 0.25, 0.75, 0.375, 0.625],
 [0.5, 0.625, 0.625, 0.375, 0.625, 0.5, 0.75, 0.875, 0.5, 0.5],
 [0.75, 0.625, 0.625, 0.5, 0.625, 0.25, 0.625, 0.625, 0.75, 0.625],
 [0.25, 0.5, 0.375, 0.5, 0.625, 0.875, 0.375, 0.5, 0.25, 0.5],
 [0.25, 0.625, 0.375, 0.375, 0.375, 0.375, 0.5, 0.5, 0.25, 0.5]]